In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#reading the file
words=open("names.txt",'r').read().splitlines()
import re
for _ in range(len(words)):
    words[_]=words[_].lower() #to convert the capital letters into small
    pattern = r'[^a-z\s]'
    words[_] = re.sub(pattern, '', words[_])
    #print(_)
words[:10]
words[:8]

['abhishek', 'aman', 'harsh', 'ayush', 'aditi', 'anjali', 'shubham', 'anushka']

In [4]:
chars=sorted(list(set(''.join(words))))
stoi={s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos={i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [38]:
#build the dataset
block_size=3 #how many character do we take to predict the next character
x,y=[],[]
for w in words:
    context=[0]*block_size
    for ch in w+'.':
        ix=stoi[ch]
        x.append(context)
        y.append(ix)
        #print(''.join(itos[i] for i in context),'---->',ch)
        context=context[1:] + [ix]
        
x=torch.tensor(x)
y=torch.tensor(y)

In [39]:
x.shape,x.dtype,y.shape,y.dtype

(torch.Size([8753, 3]), torch.int64, torch.Size([8753]), torch.int64)

In [40]:
c=torch.randn(27,2)

In [8]:
emb=c[x]
emb.shape

torch.Size([32, 3, 2])

In [9]:
w1=torch.randn(6,100)# 6 is the number of inputs and 100 is the number of neuron
b1=torch.randn(100)


In [10]:
h=torch.tanh(emb.view([-1,6])@w1+b1)
h

tensor([[-9.9997e-01, -1.0000e+00, -9.9248e-01,  ...,  9.0138e-02,
          9.9916e-01, -7.5056e-01],
        [-9.5854e-01,  9.6293e-01, -9.7228e-01,  ..., -9.2685e-01,
          5.0888e-01,  6.2081e-02],
        [ 9.5447e-01,  1.0000e+00, -9.1917e-01,  ..., -8.4217e-01,
         -9.9966e-01, -8.2513e-01],
        ...,
        [ 9.9859e-01, -9.9998e-01,  9.3714e-01,  ..., -7.0330e-01,
         -8.8679e-01, -7.9149e-01],
        [-4.3479e-01,  3.1261e-01, -7.1853e-01,  ..., -9.5796e-01,
         -2.3752e-01, -4.4581e-01],
        [-9.8899e-01, -9.7246e-01, -7.7157e-01,  ...,  4.7008e-01,
          9.9246e-01,  6.4343e-05]])

In [11]:
h.shape
#32,6@6,100 ==32,100
#32,100
#+1,100

torch.Size([32, 100])

In [12]:
w2=torch.randn(100,27)
b2=torch.randn(27)

In [13]:
logits=h@w2+b2

In [14]:
counts=logits.exp()

In [17]:
prob=counts/counts.sum(1,keepdims=True)

In [18]:
prob.shape

torch.Size([32, 27])

In [20]:
#-prob[torch.arrange(32),y] this means iterate through all the 32 row and pluck the probability of prediction at yi index
loss=-prob[torch.arange(32),y].log().mean()
loss

tensor(21.1044)

In [21]:
#--------------------now in more arranged manner--------------------------

In [51]:
x.shape,y.shape #dataset

(torch.Size([8753, 3]), torch.Size([8753]))

In [52]:
g=torch.Generator().manual_seed(2147483647)
c=torch.randn((27,2),generator=g)

w1=torch.randn((6,100),generator=g)
b1=torch.randn(100,generator=g)

w2=torch.randn((100,27),generator=g)
b2=torch.randn(27,generator=g)

# counts=logits.exp()
# prob=counts/counts.sum(1,keepdims=True)
# loss=-prob[torch.arange(32),y].log().mean()

#loss=F.cross_entropy(logits,y)# more efficient than calculating mannually because in this forward and backward pass will work more efficiently and mathematical fucntion will behave well

parameter=[c,w1,b1,w2,b2]


In [68]:
for p in parameter:
    p.requires_grad=True

In [81]:
for _ in range(10000):
    #forward pass
    ix=torch.randint(0,x.shape[0],(32,))#creating mini batch
    emb=c[x[ix]]
    h=torch.tanh(emb.view([-1,6])@w1+b1)
    logits=h@w2+b2
    loss=F.cross_entropy(logits,y[ix])
    #print(loss.item())
    #backward pass
    for p in parameter:
        p.grad=None
    loss.backward()
    for p in parameter:
        p.data+=-0.01*p.grad
        
        

In [82]:
emb=c[x]
h=torch.tanh(emb.view([-1,6])@w1+b1)
logits=h@w2+b2
loss=F.cross_entropy(logits,y)
loss

tensor(1.8108, grad_fn=<NllLossBackward0>)

In [86]:
g=torch.Generator().manual_seed(2147483647)
for i in range(15):
    out=[]
    context=[0]*block_size
    while(True):
        emb=c[torch.tensor([context])]
        h=torch.tanh(emb.view(1,-1)@w1+b1)
        logits=h@w2+b2
        counts=logits.exp()
        probs=F.softmax(logits,dim=1)
        ix=torch.multinomial(probs,num_samples=1,replacement=True,generator=g).item()
        context=context[1:]+[ix]
        out.append(ix)
        if ix==0:
            break
    print(''.join(itos[i]for i in out))

junine.
sanakar.
pratar.
akrit.
shiniti.
shas.
geet.
saheeshwanilendasshiddikrad.
shasnisheay.
arth.
farudharif.
demmarnimashwar.
sanjan.
saaya.
pranjanjani.
